In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [119]:

def calculate_cumulative_customers(total_addressable_population, stable_penetration_perc, starting_customers, midpoint_year, initial_k, min_growth_rate, years):
    stable_penetration_fixed = total_addressable_population * stable_penetration_perc
    cume_customers = []

    for i in years:
        x = starting_customers + ((stable_penetration_fixed - starting_customers) / (1 + np.exp(-initial_k * (i - midpoint_year))))
        # Apply the minimum growth rate
        if cume_customers:
            x = max(x, cume_customers[-1] * (1 + min_growth_rate))
        cume_customers.append(int(round(x)))  # Round and convert to integer
    
    return cume_customers

def calculate_new_customers(cume_customers, starting_customers):
    new_customers = []

    new_customers.append(int(cume_customers[0] - starting_customers))

    for i in range(1, len(cume_customers)):
        new_customers.append(int(cume_customers[i] - cume_customers[i - 1]))
    
    return new_customers

# Define variables
total_addressable_population = 100000
stable_penetration_perc = 0.1
starting_customers = 100
midpoint_year = 3
initial_k = 2.5
min_growth_rate = 0.05
years = np.arange(1, 11, 1)

# Calculate cumulative customers
cume_customers = calculate_cumulative_customers(total_addressable_population, stable_penetration_perc, starting_customers, midpoint_year, initial_k, min_growth_rate, years)

# Calculate new customers
new_customers = calculate_new_customers(cume_customers, starting_customers)

# Create DataFrame
customer_model = pd.DataFrame({
    'year': np.arange(0, len(cume_customers) + 1),
    'cume_customers': [starting_customers] + cume_customers,
    'new_customers': [0] + new_customers
})

# Print the DataFrame
print(customer_model)


    year  cume_customers  new_customers
0      0             100              0
1      1             166             66
2      2             851            685
3      3            5050           4199
4      4            9249           4199
5      5            9934            685
6      6           10431            497
7      7           10953            522
8      8           11501            548
9      9           12076            575
10    10           12680            604


In [120]:
# Define active rate scenarios
active_rate_scenarios = {
    '80% asymptote': [1, .8, .8, .8, .8, .8, .8, .8, .8, .8],
    '50% asymptote': [1, .5, .5, .5, .5, .5, .5, .5, .5, .5],
    '30% asymptote': [1, .3, .3, .3, .3, .3, .3, .3, .3, .3],
    '5 year decline to zero': [1, .80, .60, .40, .20, .00, .00, .00, .00, .00],
    '4 year decline to zero': [1, .75, .50, .25, .00, .00, .00, .00, .00, .00],
    '3 year decline to zdero': [1, .66, .33, .00, .00, .00, .00, .00, .00, .00],
}

In [121]:
# Select an active rate scenario
scenario_name = '80% asymptote'
active_rates = active_rate_scenarios[scenario_name]

# Initialize a list to store total active customers for each year
total_active_customers = [0] * len(df)

# Apply the active rate scenario to each cohort
for start_year in df['year']:
    for i, rate in enumerate(active_rates):
        active_year = start_year + i
        if active_year < len(total_active_customers):
            total_active_customers[active_year] += df.loc[start_year, 'new_customers'] * rate

# Round the active customers to whole integers
total_active_customers = [round(x) for x in total_active_customers]

# Create the customer_model DataFrame
customer_model['active_customers'] = total_active_customers

# Display the resulting DataFrame
print(customer_model)



    year  cume_customers  new_customers  active_customers
0      0             100              0                 0
1      1             166             66                66
2      2             851            685               738
3      3            5050           4199              4800
4      4            9249           4199              8159
5      5            9934            685              8004
6      6           10431            497              8364
7      7           10953            522              8787
8      8           11501            548              9230
9      9           12076            575              9696
10    10           12680            604             10185


In [122]:
# Define P&L variables
monthly_ARPU = 15.00
gross_margin = .9
cac = 25.00
monthly_support_per_active = 10.00
per_hc_cost = 175000
hc_cost_inflation = 0.03
other_fixed_cost_ratio = 0.05

In [127]:
headcount = [0, 5, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [137]:


customer_model['revenue'] = customer_model['active_customers'] * monthly_ARPU * 12
customer_model['COGS'] = customer_model['revenue'] * (1-gross_margin)
customer_model['gross_profit'] = customer_model['revenue'] - customer_model['COGS']
customer_model['marketing_expense'] = customer_model['new_customers'] * cac
customer_model['support_expense'] = customer_model['active_customers'] * monthly_support_per_active
customer_model['headcount'] = headcount
customer_model['headcount_expense'] = customer_model['headcount'] * (per_hc_cost * ((1+hc_cost_inflation) ** customer_model['year']))
customer_model['other_expense'] = customer_model['headcount_expense'] * other_fixed_cost_ratio
customer_model['total_operating_expense'] = customer_model['marketing_expense'] + customer_model['support_expense'] + customer_model['headcount_expense'] + customer_model['other_expense']
customer_model['operating_profit'] = customer_model['gross_profit'] - customer_model['total_operating_expense']
customer_model

,year,cume_customers,new_customers,active_customers,revenue,COGS,gross_profit,marketing_expense,support_expense,headcount,headcount_expense,other expense,other_expense,total_operating_expense,operating_profit
0,0,100,0,0,0.0,0.0,0.0,0.0,0.0,0,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
1,1,166,66,66,11880.0,1188.0,10692.0,1650.0,660.0,5,9.012500e+05,45062.500000,45062.500000,9.486225e+05,-9.379305e+05
2,2,851,685,738,132840.0,13284.0,119556.0,17125.0,7380.0,10,1.856575e+06,92828.750000,92828.750000,1.973909e+06,-1.854353e+06
3,3,5050,4199,4800,864000.0,86400.0,777600.0,104975.0,48000.0,10,1.912272e+06,95613.612500,95613.612500,2.160861e+06,-1.383261e+06
4,4,9249,4199,8159,1468620.0,146862.0,1321758.0,104975.0,81590.0,10,1.969640e+06,98482.020875,98482.020875,2.254687e+06,-9.329294e+05
5,5,9934,685,8004,1440720.0,144072.0,1296648.0,17125.0,80040.0,10,2.028730e+06,101436.481501,101436.481501,2.227331e+06,-9.306831e+05
6,6,10431,497,8364,1505520.0,150552.0,1354968.0,12425.0,83640.0,10,2.089592e+06,104479.575946,104479.575946,2.290136e+06,-9.351681e+05
7,7,10953,522,8787,1581660.0,158166.0,1423494.0,13050.0,87870.0,10,2.152279e+06,107613.963225,107613.963225,2.360813e+06,-9.373192e+05
8,8,11501,548,9230,1661400.0,166140.0,1495260.0,13700.0,92300.0,10,2.216848e+06,110842.382121,110842.382121,2.433690e+06,-9.384300e+05
9,9,12076,575,9696,1745280.0,174528.0,1570752.0,14375.0,96960.0,10,2.283353e+06,114167.653585,114167.653585,2.508856e+06,-9.381037e+05
